In [1]:
# Dependencies
!pip3 install pandas
!pip3 install scipy
from scipy.stats import ttest_ind
import pandas as pd
import numpy as np
import glob


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip


In [2]:
# Read map .csv files
mapDepthWith5FilePath = '../outputs/map@5.csv'
map_depth_with_5_df = pd.read_csv(mapDepthWith5FilePath)

mapDepthWith10FilePath = '../outputs/map@10.csv'
map_depth_with_10_df = pd.read_csv(mapDepthWith10FilePath)

mapDepthWith20FilePath = '../outputs/map@20.csv'
map_depth_with_20_df = pd.read_csv(mapDepthWith20FilePath)

In [3]:
# Filtering

aps_with_depth_5 = map_depth_with_5_df.iloc[0:-1, 1:-1]
aps_with_depth_10 = map_depth_with_10_df.iloc[0:-1, 1:-1]
aps_with_depth_20 = map_depth_with_20_df.iloc[0:-1, 1:-1]

In [10]:
significance_level = 0.05
dfs = [aps_with_depth_5, aps_with_depth_10, aps_with_depth_20]
depth_values = [5, 10, 20]

for i, df in enumerate(dfs):
    depth = depth_values[i]
    
    columns = df.columns
    system_name_list = columns
    num_columns = len(columns)
    significant_results = [[None] * num_columns for _ in range(num_columns)]
    significant_count = 0  # Counter for significant differences

    
    # Calculate the total number of system comparisons
    total_comparisons = num_columns * (num_columns - 1)

    for i in range(num_columns):
        col1 = columns[i]

        # Compare col1 with every other column
        for j in range(num_columns):
            if i == j:  # Skip self-comparisons
                significant_results[i][j] = (None, None)
            else:
                t_statistic, p_value = ttest_ind(df.iloc[:, i], df.iloc[:, j])
                is_significant_different = p_value <= significance_level
                significant_results[i][j] = (p_value, is_significant_different)

                if is_significant_different:
                    significant_count += 1

    # Create a DataFrame with the results
    significant_df = pd.DataFrame(significant_results, index=system_name_list, columns=system_name_list)

    # Save the DataFrame to a CSV file
    output_file = f"../outputs/test_map_depth_{depth}.csv"
    significant_df.to_csv(output_file)
    
    # Output the count of significant differences and total comparisons
    print(f"Depth {depth}: Number of significant differences = {significant_count} out of {total_comparisons} system comparisons")

Depth 5: Number of significant differences = 90 out of 210 system comparisons
Depth 10: Number of significant differences = 88 out of 210 system comparisons
Depth 20: Number of significant differences = 86 out of 210 system comparisons
